In [2]:
# [+] PySpark 설정
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster('local').setAppName('shuffling_and_partitioning')
sc = SparkContext(conf=conf)

In [3]:
# 데이터셋
filename = "fhvhv_tripdata_2020-03_short.csv"

In [4]:
# [+] 데이터셋 로딩 및 헤더 추출
lines = sc.textFile("./data/"+filename)
header = lines.first()

In [5]:
header

'hvfhs_license_num,dispatching_base_num,pickup_datetime,pickup_datetime,PULocationID,DOLocationID,SR_Flag,,,,,,'

In [7]:
# [+] filter()를 이용한 헤더 제거
filtered_lines = lines.filter(lambda x: x != header)

승차일자 추출과 동시에 k-v 형식으로 변환 저장
승차일자 추출 :
    .map(lambda x: x.split(",")[2].split(" ")[0])
k-v 형식으로 변환 :
    .map(lambda x: (x.split(",")[2].split(" ")[0],1))

In [8]:
# [+] map()을 이용한 승차일자 추출하기(K-V RDD로 저장)

dates = filtered_lines.map(lambda x: (x.split(",")[2].split(" ")[0],1))

In [9]:
# 첫 번째 값 출력하기, 데이터가 클 때 collect() 사용은 비효율적
dates.first()

('2020-03-01', 1)

In [15]:
"""
  일별 승차횟수 계산: reduceByKey()
"""

# [+] reduceByKey()를 이용한 일별 승차횟수 계산
reduced = dates.reduceByKey(lambda a, b: a+b)

In [17]:
# 매직 함수  python 함수 아님
#           jupyter 제공
    # %%time : 해당 셀이 걸리는 시간 측정

reduced.collect()

UsageError: Line magic function `%%time` not found.


In [18]:
"""
  일별 승차횟수 계산: groupByKey()
"""

# [+] groupByKey()를 이용한 일별 데이터 그룹핑
groups = dates.groupByKey()

In [19]:
groups.first()

('2020-03-01', <pyspark.resultiterable.ResultIterable at 0x1d7ce585b20>)

In [20]:
# [+] mapValues()를 이용한 일별 승차횟수 계산
counts = groups.mapValues(len)

In [22]:
%%time
counts.collect()

Wall time: 2.82 s


[('2020-03-01', 784246), ('2020-03-02', 264329)]

In [ ]:
# partitionBy
x = sc.parallelize([(0,1),(1,2),(2,3)],2)
y = x.partitionBy(numPartitions=3, partitionFunc=lambda x: x)
print(x.glom().collect())
print(y.glom().collect())

In [ ]:
# repartition

x = sc.parallelize([1,2,3,4,5],2)
y = x.repartition(numPartitions=3)
print(x.glom().collect())
print(y.glom().collect())

In [ ]:
# coalesce

x = sc.parallelize([1,2,3,4,5],2)
y = x.coalesce(numPartitions=1)

print(x.glom().collect())
print(y.glom().collect())